In [1]:
Vir = lie_algebras.VirasoroAlgebra(QQ)
M = Vir.verma_module(1/2, 1/2)
d = Vir.basis()
v = M.highest_weight_vector()
m2 = (-4*d[-2] + 3*d[-1]*d[-1])*v
m3 = (-12*d[-2]*d[-1] + 4*d[-1]*d[-1]*d[-1] + 3*d[-3])*v

def L_partition_vector(partition, vector):
    for i in range(len(partition)):
        vector = d[-partition[len(partition) - i - 1]]*vector
    return vector

def my_coefficient(partition, vector):
    return vector.monomial_coefficients().get(tuple(-k for k in partition), 0)

def number_of_ones(partition):
    for i in range(len(partition)):
        if partition[len(partition) - i - 1] != 1:
            return i
    return len(partition)

def my_len(partition):
    return 2*len(partition) - number_of_ones(partition)

def to_polynomial(partition):
    polynomial = [0]*(sum(partition) - 1)
    for i in range(len(partition) - number_of_ones(partition)):
        polynomial[partition[i] - 2] += 1
    return polynomial

def to_vector(row, n):
    list_of_partition = ordered_partitions(n)
    return sum(row[i]*L_partition_vector(list_of_partition[i], v) for i in range(len(row)))

def my_order(partition):
    polynomial = to_polynomial(partition)
    polynomial = [-k for k in polynomial]
    polynomial.reverse()
    return ((my_len(partition), len(partition) - number_of_ones(partition), polynomial))

def ordered_partitions(n):
    if n <= 0:
        return [[]]
    list_of_partition = list(Partitions(n))
    list_of_partition.sort(key = my_order, reverse = True)
    return list_of_partition

def matrix_UVir_m2_m3(n):
    list_of_partition = ordered_partitions(n)
    list_of_partition2 = ordered_partitions(n - 2)
    list_of_partition3 = ordered_partitions(n - 3)
    matrix_m2 = matrix({
        (i, j): my_coefficient(list_of_partition[j], L_partition_vector(list_of_partition2[i], m2))
        for j in range(len(list_of_partition))
        for i in range(len(list_of_partition2))}, sparse = False)
    matrix_m3 = matrix({
        (i, j): my_coefficient(list_of_partition[j], L_partition_vector(list_of_partition3[i], m3))
        for j in range(len(list_of_partition))
        for i in range(len(list_of_partition3))}, sparse = False)
    matrix_m2 = matrix_m2.stack(matrix_m3)
    matrix_m2 = matrix_m2.rref()
    return matrix_m2

def mW_partition(partition):
    n = sum(partition)
    k = ordered_partitions(n).index(partition)
    A_n = matrix_UVir_m2_m3(n)
    pivots = A_n.pivots()
    try:
        return to_vector(A_n.row(pivots.index(k)), n)
    except ValueError:
        return 0

def mW_list_of_partition(list_of_partition):
    return [mW_partition(partition) for partition in list_of_partition]

In [2]:
list_of_partition = [[2], [1, 1, 1], [3, 1, 1], [3, 3], [4, 3, 1], [4, 4, 1], [5, 4, 1, 1],
                     [6, 5, 3, 1]]
%time mW_list_of_partition(list_of_partition)

CPU times: user 1min 35s, sys: 12.3 ms, total: 1min 35s
Wall time: 1min 35s


[d[-2]*v - 3/4*d[-1]*d[-1]*v,
 -3*d[-3]*v + d[-1]*d[-1]*d[-1]*v,
 -3/2*d[-5]*v - d[-4]*d[-1]*v + d[-3]*d[-1]*d[-1]*v,
 -2*d[-6]*v - 3/2*d[-5]*d[-1]*v + 1/3*d[-4]*d[-1]*d[-1]*v + d[-3]*d[-3]*v,
 -9/2*d[-8]*v - 9/2*d[-7]*d[-1]*v - d[-6]*d[-1]*d[-1]*v + 3/2*d[-5]*d[-3]*v + d[-4]*d[-4]*v + d[-4]*d[-3]*d[-1]*v,
 -27/4*d[-9]*v - 63/8*d[-8]*d[-1]*v - 19/8*d[-7]*d[-1]*d[-1]*v + 3/4*d[-6]*d[-3]*v + 21/8*d[-5]*d[-4]*v + 9/8*d[-5]*d[-3]*d[-1]*v + d[-4]*d[-4]*d[-1]*v,
 -393/16*d[-11]*v - 117/4*d[-10]*d[-1]*v - 189/16*d[-9]*d[-1]*d[-1]*v - 39/8*d[-8]*d[-3]*v + 27/8*d[-7]*d[-4]*v + 9/8*d[-7]*d[-3]*d[-1]*v + 27/4*d[-6]*d[-5]*v + 9/2*d[-6]*d[-4]*d[-1]*v + 21/8*d[-5]*d[-5]*d[-1]*v + d[-5]*d[-4]*d[-1]*d[-1]*v,
 -4977/20*d[-15]*v - 12207/40*d[-14]*d[-1]*v - 140449/960*d[-13]*d[-1]*d[-1]*v - 19159/160*d[-12]*d[-3]*v - 3973/160*d[-11]*d[-4]*v - 3681/160*d[-11]*d[-3]*d[-1]*v + 567/80*d[-10]*d[-5]*v - 1/12*d[-10]*d[-4]*d[-1]*v + 131/5*d[-9]*d[-6]*v + 369/20*d[-9]*d[-5]*d[-1]*v + 81/20*d[-9]*d[-4]*d[-1]*d[-1]